In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 5.6MB/s 
     |████████████████████████████████| 1.5MB 20.7MB/s 
     |████████████████████████████████| 1.1MB 53.4MB/s 
     |████████████████████████████████| 2.9MB 59.8MB/s 
     |████████████████████████████████| 890kB 60.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.0-cp36-none-any.whl size=102655 sha256=0ba6f9ec03fad60b1f1834085c036059c531336f5d03494fafcdf4d12b47f775
  Stored in directory: /root/.cache/pip/wheels/ff/76/65/50258d8b7930e909ea2f5bd006a23d520a16765af13ab45bb3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5d28cc3d52886f856e2c7068d6080bd3b367a3deaeea7db2bedabef543929e7d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv
from zipfile import ZipFile

from sentence_transformers import SentenceTransformer, util
model_path = 'monologg/koelectra-base-v3-discriminator'
embedder = SentenceTransformer(model_path)

def preprocess(text):
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'[\x03,\xad]', '', text)
    #text = re.sub('[0-9]+', '', text)
    #text = re.sub(r'\s*[一-龥]+', '', text)
    text = re.sub(r'[-=+,___·#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;}{∼＃°？▶▲▼◀★♥■]', '',text)
    text=" ".join(text.split())
    return text

df = pd.read_csv("/content/drive/My Drive/dacon/news_train.csv") # train.csv 불러오기
test = pd.read_csv("/content/drive/My Drive/dacon/news_test.csv") # test.csv 불러오기
df['clean_text'] = df['content'].apply(preprocess)
test['clean_text'] = test['content'].apply(preprocess)
df['clean_title'] = df['title'].apply(preprocess)
test['clean_title'] = test['title'].apply(preprocess)
train=df.sample(frac=0.85,random_state=200) #0.9
val=df.drop(train.index).sample(frac=1.0)
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO, handlers=[LoggingHandler()])

#### data aug   *info(bool)= 0 or 1
train_samples, val_samples, test_samples = [], [], []
train_text = train['clean_text'].values
train_title = train['clean_title'].values
train_y =  train['info'].values
val_text = val['clean_text'].values
val_title = val['clean_title'].values
val_y = val['info'].values
test_text = test['clean_text'].values
test_title = test['clean_title'].values

Exception when trying to download https://sbert.net/models/monologg/koelectra-base-v3-discriminator.zip. Response 404


In [ ]:
corpus = test[138555:138583]['clean_text'].values
len(corpus)

28

In [ ]:
corpus

In [ ]:
for row in range(len(train_text)):
    sample = InputExample(texts=[str(train_text[row]),str(train_title[row])], 
                          label=int(train_y[row]))  
    train_samples.append(sample)

for row in range(len(val_text)):
    sample = InputExample(texts=[str(val_text[row]),str(val_title[row])], 
                          label=int(val_y[row]))  
    val_samples.append(sample)
  
test1 = [[test_text[i]] for i in range(0, len(test_text))]
test2 = [[test_title[i],test_text[i]] for i in range(0, len(test_text))]

In [ ]:
#xlm-r-large-en-ko-nli-ststb
#'monologg/koelectra-base-v3-discriminator'  
#ce-roberta-large-stsb
#kykim/electra-kor-base
#Configuration
train_batch_size = 2 #16,32
num_epochs = 2
model = 'sentence-transformers/ce-roberta-large-stsb'
model = CrossEncoder(model, num_labels=1) #,device='cuda'
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
evaluator = CEBinaryClassificationEvaluator.from_input_examples(val_samples)

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.15) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

model_save_path='content/drive/MyDrive/dacon'
# Train the model
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=5000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

2020-12-29 10:15:21 - Lock 140244594522208 acquired on /root/.cache/huggingface/transformers/8690e4e148d29674545a2e355563654b303447edaa6946fb8a563bf22e491fc7.603e50b2b1f7ad6d39458147ee15348e9fe9136812c012306b6e6dba4e19ddf4.lock



2020-12-29 10:15:21 - Lock 140244594522208 released on /root/.cache/huggingface/transformers/8690e4e148d29674545a2e355563654b303447edaa6946fb8a563bf22e491fc7.603e50b2b1f7ad6d39458147ee15348e9fe9136812c012306b6e6dba4e19ddf4.lock
2020-12-29 10:15:21 - Lock 140241671787184 acquired on /root/.cache/huggingface/transformers/9daa1fdaed414c76619e89ca392039f8dd593d2bb3c6a20e2f1db47915f6cb7e.5d9f52e01d76409c5be4deb8854f02bec491bd198e76c090c11f02ee38b0f781.lock



2020-12-29 10:15:47 - Lock 140241671787184 released on /root/.cache/huggingface/transformers/9daa1fdaed414c76619e89ca392039f8dd593d2bb3c6a20e2f1db47915f6cb7e.5d9f52e01d76409c5be4deb8854f02bec491bd198e76c090c11f02ee38b0f781.lock
2020-12-29 10:15:58 - Lock 140241603371752 acquired on /root/.cache/huggingface/transformers/8bbb1f2a6f0be69baf185ddfda97e96a124a007579796190d745d59240d10e80.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock



2020-12-29 10:15:58 - Lock 140241603371752 released on /root/.cache/huggingface/transformers/8bbb1f2a6f0be69baf185ddfda97e96a124a007579796190d745d59240d10e80.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock
2020-12-29 10:15:58 - Lock 140241603451424 acquired on /root/.cache/huggingface/transformers/88ee0e5064a66b2bf52840f90977ad14ee35012cd0f73707d50a4f97f4999a4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock



2020-12-29 10:15:59 - Lock 140241603451424 released on /root/.cache/huggingface/transformers/88ee0e5064a66b2bf52840f90977ad14ee35012cd0f73707d50a4f97f4999a4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
2020-12-29 10:15:59 - Lock 140241603373320 acquired on /root/.cache/huggingface/transformers/be1de0e10a79df0c431000156524f28247c8ac60dde79ffc217c5205fe9ce4d7.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock



2020-12-29 10:15:59 - Lock 140241603373320 released on /root/.cache/huggingface/transformers/be1de0e10a79df0c431000156524f28247c8ac60dde79ffc217c5205fe9ce4d7.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
2020-12-29 10:15:59 - Lock 140241603418432 acquired on /root/.cache/huggingface/transformers/9008b365fcc56710190d89a30481369d582784402260092c92d65d4621897e29.33dce59447abe5faeaa635b4c89885dd47444800e6459f3154dd10e294fe3447.lock



2020-12-29 10:15:59 - Lock 140241603418432 released on /root/.cache/huggingface/transformers/9008b365fcc56710190d89a30481369d582784402260092c92d65d4621897e29.33dce59447abe5faeaa635b4c89885dd47444800e6459f3154dd10e294fe3447.lock
2020-12-29 10:15:59 - Use pytorch device: cuda
2020-12-29 10:15:59 - Warmup-steps: 15141


2020-12-29 10:51:26 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 0 after 5000 steps:
2020-12-29 11:13:58 - Accuracy:           93.17	(Threshold: 0.1304)
2020-12-29 11:13:58 - F1:                 91.52	(Threshold: 0.1187)
2020-12-29 11:13:58 - Precision:          89.49
2020-12-29 11:13:58 - Recall:             93.65
2020-12-29 11:13:58 - Average Precision:  96.47

2020-12-29 11:13:58 - Save model to content/drive/MyDrive/dacon


In [ ]:
model.save_pretrained('cross_encoder1')  

2020-12-29 07:38:24 - Save model to cross_encoder1


In [ ]:
scores1 = model.predict(test1,batch_size=16)

In [ ]:
scores2 = model.predict(test2,batch_size=16)

In [ ]:
df_describe = pd.DataFrame(data = [scores1,scores2]).T
df_describe.describe()

,0,1
count,142565.000000,142565.000000
mean,0.382605,0.392593
std,0.485038,0.485687
min,0.000005,0.000005
25%,0.000006,0.000009
50%,0.000007,0.000051
75%,0.999995,0.999988
max,0.999996,0.999996


In [ ]:
#x1 = [1 if scores[i]>0.5 else 0 for i in range(len(scores))]
#x2 = [1 if scores[i]>0.5 else 0 for i in range(len(scores1))]
count = 0
l1 = []
for i in range(len(scores)):
  if abs(scores[i]-scores1[i])>0.9:
    count += 1
    l1.append(i)
print(count)

6076


In [ ]:
sample_sub=pd.read_csv('/content/drive/My Drive/dacon/sample_submission.csv')
x2 = [1 if scores1[i]>0.5 else 0 for i in range(len(scores1))]
sample_sub['info']=x2
sample_sub.to_csv('/content/drive/My Drive/dacon/encoder_train_text1.csv',index=False)

In [ ]:
sample_sub

,id,info
0,NEWS00237_1,1
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
